In [8]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
import evaluate

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [11]:
path = 'data/'

val_table = {'article': [], 'abstract': []}

table = pd.read_parquet(path + 'validation-00000-of-00001.parquet')
for i in table.sample(len(table) - 6000).values:
    if i[0] != '':
        val_table['article'].append(i[0])
        val_table['abstract'].append(i[1])

In [14]:
def chunk_text(text, tokenizer, max_length=512):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    # Декодируем каждый чанк обратно в текст
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


In [16]:
predictions = []

c = 0

for i in val_table['article']:
    all_chunks = []
    chunks = chunk_text(i, tokenizer)

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True,
                           padding=True, max_length=1024).to(device)

        outputs = model.generate(**inputs, max_length=150)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        all_chunks.append(generated_text)

    result = " ".join(all_chunks)
    predictions.append(result)

    c += 1
    print(c)

predictions

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


['Conrad et al. ( 2009) show that allergic risk can also be modulated by microbial exposure before birth. mice born to dams exposed to bacteria during pregnancy were less likely to develop allergic responses than those born to unexposed mothers. infants from atopic families who were at high risk for developing allergic disease had reduced numbers of immunocompetent t cells in the circulation. This gut - centric view has been challenged by findings from landmark human studies relating to asthma and allergy in traditional farming communities. These epidemiological studies defined a unique gradient in allergy and asthma prevalence between children born into traditional farming families and their rural and urban counterparts. A major international research effort has now developed to characterize the underlying immunological processes. 5 - 13-yr - old children of farm - exposed mothers displayed reduced rates of sensitization and increased tlr2, tlr4, and cd14 gene expression in pbmcs. The

In [19]:
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=predictions, references=val_table['abstract'])

In [20]:
results

{'rouge1': 0.3530235424992888,
 'rouge2': 0.13757168879872766,
 'rougeL': 0.18883325365354148,
 'rougeLsum': 0.27314085922630915}